# **Employee Satisfaction Survey Data**


### 1. Importing Relevant Libraries


In [64]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import  train_test_split
from sklearn.metrics import mean_squared_error



### 2. Reading and Analyzing the Dataframe

In [65]:
satisfaction_data = pd.read_csv("Employee Attrition.csv")

In [66]:
satisfaction_data.head(10)

,Emp ID,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,1.0,0.38,0.53,2.0,157.0,3.0,0.0,0.0,sales,low
1,2.0,0.80,0.86,5.0,262.0,6.0,0.0,0.0,sales,medium
2,3.0,0.11,0.88,7.0,272.0,4.0,0.0,0.0,sales,medium
3,4.0,0.72,0.87,5.0,223.0,5.0,0.0,0.0,sales,low
4,5.0,0.37,0.52,2.0,159.0,3.0,0.0,0.0,sales,low
5,6.0,0.41,0.50,2.0,153.0,3.0,0.0,0.0,sales,low
6,7.0,0.10,0.77,6.0,247.0,4.0,0.0,0.0,sales,low
7,8.0,0.92,0.85,5.0,259.0,5.0,0.0,0.0,sales,low
8,9.0,0.89,1.00,5.0,224.0,5.0,0.0,0.0,sales,low
9,10.0,0.42,0.53,2.0,142.0,3.0,0.0,0.0,sales,low


In [67]:
satisfaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15787 entries, 0 to 15786
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Emp ID                 14999 non-null  float64
 1   satisfaction_level     14999 non-null  float64
 2   last_evaluation        14999 non-null  float64
 3   number_project         14999 non-null  float64
 4   average_montly_hours   14999 non-null  float64
 5   time_spend_company     14999 non-null  float64
 6   Work_accident          14999 non-null  float64
 7   promotion_last_5years  14999 non-null  float64
 8   dept                   14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(8), object(2)
memory usage: 1.2+ MB


### 3. Dealing with Missing Values 

In [68]:
satisfaction_data.isna().sum()

Emp ID                   788
satisfaction_level       788
last_evaluation          788
number_project           788
average_montly_hours     788
time_spend_company       788
Work_accident            788
promotion_last_5years    788
dept                     788
salary                   788
dtype: int64

We see 788 null values within this dataset. Let's clean this up and remove these rows.

In [69]:
satisfaction_data = satisfaction_data.dropna(axis = 0)
satisfaction_data.isna().sum()

Emp ID                   0
satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
promotion_last_5years    0
dept                     0
salary                   0
dtype: int64

# 4. Plotting the Data #

Next, let's plot the boxplots for each feature within the dataset to help determine the overall spread,  mean, median,  as well as identifying any outliers with the data. We detected outliers for the "Time spent at company", "Work Accident", and the "Promotion the last 5 years" features. 


In [70]:
figure = make_subplots(rows = 1, cols = len(satisfaction_data.columns))
for i, col in enumerate(satisfaction_data.select_dtypes(include=['int64', 'float64']).columns, 1):
    figure.add_trace(go.Box(y = satisfaction_data[col], name = col), row = 1, col = i)
figure.update_layout(width = 1600, height = 600, title_text = "Satisfaction Data Boxplots", showlegend = False)
figure.show()

Next, let's look at the distribution of data for the features that contain outliers (Work Accident Count, Number of Promotions, and Tenure at Company). 

In [71]:
work_accident_count = satisfaction_data['Work_accident'].value_counts()
figure = go.Figure(go.Bar(x = work_accident_count.index, y = work_accident_count.values))
figure.update_layout(width = 500, height = 400,
    title="Work Accident Counts",
    xaxis_title="Work Accident",
    yaxis_title="Number of Work Accidents", 
    xaxis = dict(tickmode = 'array',
                 tickvals = [0, 1], 
                 ticktext = ["no", "yes"]))


In [72]:
promotions = satisfaction_data['promotion_last_5years'].value_counts()
figure = go.Figure(go.Bar(x = promotions.index, y = promotions.values))
figure.update_layout(width = 500, height = 400,
    title="Promotions over the Last Five Years",
    xaxis_title="Promotion",
    yaxis_title="Count", 
    xaxis = dict(tickmode = 'array',
                 tickvals = [0, 1], 
                 ticktext = ["no", "yes"]))

In [73]:
time_spent = satisfaction_data['time_spend_company'].value_counts()
figure = go.Figure(go.Bar(x = time_spent.index, y = time_spent.values))
figure.update_layout(width = 600, height = 450,
    title="Years Spent at Company",
    xaxis_title="Years",
    yaxis_title="Count", 
    xaxis = dict(tickmode = 'array',
                 tickvals = [2, 3, 4, 5, 6, 7, 8, 9, 10], 
                 ))

Looking at the plots for "Work Accident Counts" and "Promotions", the outliers appear to respondents who have responded "yes". Given the significantly uneven distribution of respondents, these two features likely will not be meaningful predictors for employee satisfaction. 

It is apparent from looking at the data that the vast majority of respondents have worked at the company over 2 - 4 years. This would explain the outliers seen, and would significantly skew the weightage of this feature towards respondents who have not been at the company for a long time. 

# 5. Label Encoding and Finding Correlation Values

Next we want to observe any relationships between each feature in the dataset and the employee satisfaction level. We must first use label encoding to convert the categorical variables (Salary and Department) into numeric values. 

In [74]:
print(satisfaction_data['salary'].unique())
print(satisfaction_data['dept'].unique())


['low' 'medium' 'high']
['sales' 'accounting' 'hr' 'technical' 'support' 'management' 'IT'
 'product_mng' 'marketing' 'RandD']


In [75]:
Le = LabelEncoder()
satisfaction_data['dept']= Le.fit_transform(satisfaction_data['dept'])
satisfaction_data['salary']=Le.fit_transform(satisfaction_data['salary'])
satisfaction_data.head(5)

,Emp ID,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,1.0,0.38,0.53,2.0,157.0,3.0,0.0,0.0,7,1
1,2.0,0.80,0.86,5.0,262.0,6.0,0.0,0.0,7,2
2,3.0,0.11,0.88,7.0,272.0,4.0,0.0,0.0,7,2
3,4.0,0.72,0.87,5.0,223.0,5.0,0.0,0.0,7,1
4,5.0,0.37,0.52,2.0,159.0,3.0,0.0,0.0,7,1


Next, we plot the correlation data using the pearson method. 

In [76]:
figure = px.imshow(satisfaction_data.corr(method = 'pearson'), text_auto = True)
figure.update_layout(width = 1000, height = 1000, title_text = "Feature Correlation Heatmap", showlegend = False)
figure.show()

Based on the above findings, none of the above features have a significantly high correlation to employee satisfaction level by itself. We see the highest correlation values (positive and negative) with the "Last Evaluation", "Number of Projects", and "Time spent at company" features. We also see that salary, department, work accidents, promotions and average monthly hours seem to have a minimal correlation. 

Note: Employee ID is disregarded in this data set, as they are unique identifiers and have no meaning towards employee satisfaction. 

# 6. Training the Models, and Displaying the Results

Finally, we split the data into train and test sets, build and train the models, and display the results. We used the following models and their respective parameters:

1. Linear Regression
2. Polynomial Regression (degree = 3)
3. Ridge Regression (alpha = 1.0, degree = 3)
4. Lasso Regression (alpha = 1.0, degree = 3)

All of the polynomial based regression models (polynomial, ridge, and lasso) fit a degree 3 polynomial to reduce computational complexity. Since there is a wide variation in the numeric data ranges for all of the features, we are implementing a standard scaler to scale the data. We are using a training size of 80 percent, and a test data size of 20 percent.  

In [77]:
y = satisfaction_data['satisfaction_level']
X = satisfaction_data.drop(columns = ['satisfaction_level', 'Emp ID'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
pipeline_list = {'Linear Regression' : Pipeline([('scaler', StandardScaler()), 
                           ('linreg', LinearRegression())]),
                           'Polynomial Regression': Pipeline([('poly_features', PolynomialFeatures(degree = 3)),
                           ('scaler', StandardScaler()), 
                           ('linreg', LinearRegression())]),
                           'Ridge Regression' : Pipeline([('poly_features', PolynomialFeatures(degree = 3)),
                           ('scaler', StandardScaler()), 
                           ('ridge', Ridge())]),
                           'Lasso Regression' : Pipeline([('poly_features', PolynomialFeatures(degree = 3)),
                           ('scaler', StandardScaler()), 
                           ('lasso', Lasso())])}

Fitting the models and displaying the results: 

In [79]:

print("Model Mean Squared Errors: \n")
for key, pipeline in pipeline_list.items():
    pipeline.fit(X_train, y_train)
    y_predicted_train = pipeline.predict(X_train)
    y_predicted_test = pipeline.predict(X_test)
    train_mse = mean_squared_error(y_train, y_predicted_train)
    test_mse = mean_squared_error(y_test, y_predicted_test)
    model_accuracy = pipeline.score(X_test, y_test)
    print(key + ': ')
    print('Train MSE: %f' % train_mse)
    print('Test MSE: %f' % test_mse)
    print("Model Accuracy: %f" % model_accuracy)
    print("\n")



Model Mean Squared Errors: 

Linear Regression: 
Train MSE: 0.058280
Test MSE: 0.057525
Model Accuracy: 0.058566




Polynomial Regression: 
Train MSE: 0.037811
Test MSE: 0.041248
Model Accuracy: 0.324950


Ridge Regression: 
Train MSE: 0.037991
Test MSE: 0.041272
Model Accuracy: 0.324561


Lasso Regression: 
Train MSE: 0.061990
Test MSE: 0.061104
Model Accuracy: -0.000003




Based on the above findings, the Polynomial regression model contained the lowest MSE, and the highest model accuracy of all of the tested models. We want to explore the Ridge Regression model further, testing different values for α to find the best fit within this model. 

In [80]:
ridge_alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000 ]

ridge_mses = []
ridge_scores = []

for alpha in ridge_alphas:
    ridge_pipe = Pipeline([('poly_features', PolynomialFeatures(degree = 3)),
                           ('scaler', StandardScaler()), 
                           ('ridge', Ridge(alpha = alpha))])
    ridge_pipe.fit(X_train, y_train)
    y_predicted_train = ridge_pipe.predict(X_train)
    y_predicted_test = ridge_pipe.predict(X_test)
    train_mse = mean_squared_error(y_train, y_predicted_train)
    test_mse = mean_squared_error(y_test, y_predicted_test)
    ridge_mses.append(test_mse)
    model_accuracy = ridge_pipe.score(X_test, y_test)
    ridge_scores.append(model_accuracy)
    print("alpha: ", alpha)
    print('Train MSE: %f' % train_mse)
    print('Test MSE: %f' % test_mse)
    print("Model Accuracy: %f" % model_accuracy)
    print("\n")



alpha:  0.001
Train MSE: 0.037811
Test MSE: 0.041248
Model Accuracy: 0.324953


alpha:  0.01
Train MSE: 0.037811
Test MSE: 0.041247
Model Accuracy: 0.324978


alpha:  0.1
Train MSE: 0.037820
Test MSE: 0.041239
Model Accuracy: 0.325105


alpha:  1
Train MSE: 0.037991
Test MSE: 0.041272
Model Accuracy: 0.324561


alpha:  10
Train MSE: 0.038563
Test MSE: 0.041406
Model Accuracy: 0.322366


alpha:  100
Train MSE: 0.039327
Test MSE: 0.041695
Model Accuracy: 0.317643


alpha:  1000
Train MSE: 0.042286
Test MSE: 0.043773
Model Accuracy: 0.283625




In [81]:
figure = make_subplots(rows = 1, cols = 2, subplot_titles = ("Mean Squared Error of selected Ridge Regression Alphas", "Ridge Pipeline Accuracy"))

figure.add_trace(go.Scatter(y = ridge_mses, mode = 'lines+markers'), row = 1, col = 1)
figure.update_layout(width = 1000, height = 400,
    xaxis_title="Alpha = ",
    xaxis2_title = "Alpha = ",
    yaxis_title="Mean Squared Error",
    yaxis2_title = "Score",
    xaxis = dict(tickmode = 'array',
                 tickvals = list(range(0, 7)),
                 ticktext = ["0.001", "0.01", "0.1", "1", "10", "100", "1000" ]),
    xaxis2 = dict(tickmode = 'array',
                 tickvals = list(range(0, 7)),
                 ticktext = ["0.001", "0.01", "0.1", "1", "10", "100", "1000" ]),
                 showlegend = False)

figure.add_trace(go.Scatter(y = ridge_scores, mode = 'lines+markers', line=dict(color = 'red')), row = 1, col = 2)


We have determined that the default __(α = 0.1)__ is the best fit for the ridge regression model. This model has a higher accuracy than a Polynomial Regression model with a degree of 3.  

# 7. Summary

In this analysis, we have visually determined outliers and characteristics of the dataset features, plotted correlations, and compared the performance of different regression models. We accurately determined that the Work accidents, promotions over the last five years, average montly hours at work, and the salary had minimal impacts in predicting satisfaction.

While training and testing the models, the highest R^2 accuracy score we have obtained is __32.5 percent__ for a ridge regression model with an α value of 0.1. In the future, we can use even more advanced regressors and classifiers such as decision tree and random forest algorithms to build an even more robust and accurate predictive model. 